Why have I created this repo? I like to read a lot; papers, blog posts, twitter threads, notebooks, you name it. When I read, i've been quite well at documenting this through my blog. The blog let's me sement my thoughts, improve retention, and log what i've read. it can be quite tedious work, and sometimes I simply don't have the time, or rather capacity to do it, but i've been quite good at it, at least up until a couple of months ago. now, even though I sometimes write quite extensive summaries or thoughts about papers, i've been pretty poor at looking back at my notes when i need to remember something in a paper i read, and i attribute this mainly to a lack of **indexing**. the process often looks like this: "ooh, i know i've read something about X in a paper" *scrolls through all [blog entries](https://leonericsson.github.io/indexer), not finding what i'm looking for immediately and just give up*. yes, i could just become more patient and keep looking, why not instead take this opportunity and build something! this is the motivation for this repo. 

my initial plan. before doing any valuable research, here's what i **think** is the way i want to solve this problem, including some questions that i yet don't know the answer to:

1. save the link to every piece of research content i've read, that i can recall and find (i've already got a decent chunk saved).
2. download the content.
    - should everything be standardized into a format?
    - do i need to be careful to exclude irrelevant information from the source?
    - how do i deal with twitter threads?
3. decide on an embedding model
    - what is a good context size? relates to how we plan to chunk a 20 page paper.
4. embed content
    - how do we deal with long format texts e.g. papers?
    - does the search engine need to rely on more than just a single embedding model?
5. quantize embeddings to int8
    - does this process impact the selected embedding model?
5. index the embeddings
6. re-ranker
7. search engine
     - how do we link the source content + source url to the search results? especially
     considering that we will surely have multiple embedding points from the same source content.
    
okay, this seems like a good place to start don't you think?

**comparing embedding models**

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sentence_transformers.quantization import quantize_embeddings

# 1. Specify preffered dimensions
dimensions = 512

# 2. load model
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1", truncate_dim=dimensions)

# For retrieval you need to pass this prompt.
query = 'Represent this sentence for searching relevant passages: are all layers in a transformer equally important?'

docs = [
    query,
    """Within the field of vector search, an intriguing development has arisen: binary vector search. This approach shows promise in tackling the long-standing issue of memory consumption by achieving a remarkable 30x reduction. However, a critical aspect that sparks debate is its effect on accuracy.

We believe that using binary vector search, along with specific optimization techniques, can maintain similar accuracy. To provide clarity on this subject, we showcase a series of experiments that will demonstrate the effects and implications of this approach.""",
    """We empirically study a simple layer-pruning strategy for popular families of openweight pretrained LLMs, finding minimal degradation of performance on different
question-answering benchmarks until after a large fraction (up to half) of the layers
are removed. To prune these models, we identify the optimal block of layers to
prune by considering similarity across layers; then, to “heal” the damage, we
perform a small amount of finetuning. In particular, we use parameter-efficient
finetuning (PEFT) methods, specifically quantization and Low Rank Adapters
(QLoRA), such that each of our experiments can be performed on a single A100
GPU. From a practical perspective, these results suggest that layer pruning methods
can complement other PEFT strategies to further reduce computational resources of
finetuning on the one hand, and can improve the memory and latency of inference
on the other hand. From a scientific perspective, the robustness of these LLMs
to the deletion of layers implies either that current pretraining methods are not
properly leveraging the parameters in the deeper layers of the network or that the
shallow layers play a critical role in storing knowledge.""",
    """In this work, we introduce a significant 1-bit LLM variant called BitNet b1.58, where every parameter
is ternary, taking on values of {-1, 0, 1}. We have added an additional value of 0 to the original 1-bit
BitNet, resulting in 1.58 bits in the binary system. BitNet b1.58 retains all the benefits of the original
1-bit BitNet, including its new computation paradigm, which requires almost no multiplication
operations for matrix multiplication and can be highly optimized. Additionally, it has the same energy
consumption as the original 1-bit BitNet and is much more efficient in terms of memory consumption,
throughput and latency compared to FP16 LLM baselines.""",
]

# 2. Encode
embeddings = model.encode(docs)

# Optional: Quantize the embeddings
binary_embeddings = quantize_embeddings(embeddings, precision="ubinary")

similarities = cos_sim(embeddings[0], embeddings[1:])
print('similarities:', similarities)


In [ ]:
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer

matryoshka_dim = 512

model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)

sentences = [
    """search_document: Within the field of vector search, an intriguing development has arisen: binary vector search. This approach shows promise in tackling the long-standing issue of memory consumption by achieving a remarkable 30x reduction. However, a critical aspect that sparks debate is its effect on accuracy.

We believe that using binary vector search, along with specific optimization techniques, can maintain similar accuracy. To provide clarity on this subject, we showcase a series of experiments that will demonstrate the effects and implications of this approach.""",
    """search_document: We empirically study a simple layer-pruning strategy for popular families of openweight pretrained LLMs, finding minimal degradation of performance on different
question-answering benchmarks until after a large fraction (up to half) of the layers
are removed. To prune these models, we identify the optimal block of layers to
prune by considering similarity across layers; then, to “heal” the damage, we
perform a small amount of finetuning. In particular, we use parameter-efficient
finetuning (PEFT) methods, specifically quantization and Low Rank Adapters
(QLoRA), such that each of our experiments can be performed on a single A100
GPU. From a practical perspective, these results suggest that layer pruning methods
can complement other PEFT strategies to further reduce computational resources of
finetuning on the one hand, and can improve the memory and latency of inference
on the other hand. From a scientific perspective, the robustness of these LLMs
to the deletion of layers implies either that current pretraining methods are not
properly leveraging the parameters in the deeper layers of the network or that the
shallow layers play a critical role in storing knowledge.""",
    """search_document: In this work, we introduce a significant 1-bit LLM variant called BitNet b1.58, where every parameter
is ternary, taking on values of {-1, 0, 1}. We have added an additional value of 0 to the original 1-bit
BitNet, resulting in 1.58 bits in the binary system. BitNet b1.58 retains all the benefits of the original
1-bit BitNet, including its new computation paradigm, which requires almost no multiplication
operations for matrix multiplication and can be highly optimized. Additionally, it has the same energy
consumption as the original 1-bit BitNet and is much more efficient in terms of memory consumption,
throughput and latency compared to FP16 LLM baselines.""",
]
embeddings = model.encode(sentences, convert_to_tensor=True)
embeddings = F.layer_norm(embeddings, normalized_shape=(embeddings.shape[1],))
#embeddings = embeddings[:, :matryoshka_dim]
#embeddings = F.normalize(embeddings, p=2, dim=1)

print(embeddings)


In [ ]:
query = "search_query: are all layers in a transformer equally important?"
query_embedding = model.encode(query, convert_to_tensor=True)

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

model = SentenceTransformer(
    "jinaai/jina-embeddings-v2-base-en", # switch to en/zh for English or Chinese
    trust_remote_code=True
)

# control your input sequence length up to 8192
model.max_seq_length = 1024

embeddings = model.encode([
    'How is the weather today?',
    'What is the current weather like today?'
])
print(cos_sim(embeddings[0], embeddings[1]))


I spent the last 30 minutes comparing embedding models before realizing that this doesn't matter right now. we'll come back to this decision when we have a framework to populate the embedding space, at this stage any decent model will do. i'll leave the links to the prospects for future reference

1. https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1
2. https://huggingface.co/jinaai/jina-embeddings-v2-base-en
3. https://huggingface.co/nomic-ai/nomic-embed-text-v1.5

for no specific reason let's move forward with mixedbread's embedder for now..


---

now that we've settled on an embedding model, the next major consideration is how to handle the varying content lengths of our documents. i want to be very specific in my search queries, far more so than embedding only the abstract of a paper allows. for instance, i know that the llama 2 paper contains detailed ablations comparing mqa, gqa, and mha, so i definitely want to be able to query for those specific ablations. this raises the question of what granularity of document chunking is required. to which i currently don't know the answer, but we'll investigate this together! starting of with the most naive approach; just embed the entire document!